## Berechnung eines Steinschlags an der Kantonsstrasse in Schiers (Kt. GR) 
   In der folgenden Berechnung wird anhand der Daten der letzten drei Monate die Wahrscheinlichkeit 
   berechnet für einen tödlichen Steinschlag auf der Strasse. Anhand dessen wird ausgesagt 
   ob die Strasse gesperrt werden muss oder nicht.
   Wird die Strasse gesperrt wird untersucht ob diese den ganzen Tag gesperrt bleibt 
   oder nur für eine gewisse Zeit am Tag

### Annahmen:
1. Die Ablösungszonen 1 und 2 befinden sich an zwei verschiedenen Stellen
2. Die Steine aus den beiden Ablösungszonen werden mit dem gleichen Netz aufgefangen
3. Die Wahrscheinlichkeiten werden pro Stundenfenster berechnet und nicht pro Minute
4. Die Verteilung des Verkehrs basiert auf den Erkentnissen des Bundesamtes für Statistik
5. Steinmassen die nicht möglich sind, werden auf die nächsthöhere Skalierung gerundet (Schätzfehler)
6. Die Distanz der Gefahrenzone beträgt 20 m
7. Wenn die Gefahrenzone 20 Meter beträgt, dann muss betrachtet werden wie lange das Auto in der Gefahrenzone ist. 
8. Die Annahme ist, dass bei einem Kontakt von Auto und Stein ein Todesfall verbucht wird.
9. Die Länge des Autos ist 5 m

### Berechnungsmodel:
Um eine Simulations des Models zu bewerkstelligen müssen die einzelnen Variabeln und deren logische Abhängigkeiten 
in eine Reihenfolge gebracht werden. Aus dieser Betrachtung sollte sich dann ein Model ergeben welches durch eine 
Monte Carlo Simulation berechnet werden kann:

1. P (kinetische Energie) = Wahrscheinlichkeit Geschwindigkeit des Steins  * Wahrscheinlich Masse des Steins
2. P (T bis zum nächsten Abschlag) = Wahrscheinlichkeit des Zeitdeltas
3. P (Netz bricht beim ersten Aufprall) = P (kinetische Energie > 1000 kJ)
4. P1 (Netz bricht über Zeit) = Wenn Summe Masse der Steine > 2000 kg, dann P (kinetische Energie >= 500 kJ )
4. P2 (Netz bricht über Zeit) = Wenn Summe Masse der Steine < 2000 kg, dann P (kinetische Energie >= 1000 kJ )
6. Summe (Auto im Zeitfenster) = Summe der Menge der Autos in den Zeitfenstern (Leerung + P (Zeitdelta)  
7. Aufenthaltsdauer 1 Autos in Gefahrenzone = Distanz / Geschwindigkeit

Das Zusammensetzen der einzelnen Wahrscheinlichkeiten wird in einer Monte Carlo Simulation durch Zufallsvariablen 
simuliert

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import random
from fitter import Fitter

## 1. Daten einlesen, säubern und in eine einheitliche Form bringen.

In [ ]:
# Daten laden
raw_data_1 = pd.read_csv("out_1.csv")
raw_data_2 = pd.read_csv("out_2.csv")

In [ ]:
# Überprüfen wie viele "NA" Werte noch vorhanden sind
raw_data_1.isna().sum()

In [ ]:
raw_data_2.isna().sum()

In [ ]:
# Alle Zeilen löschen die in ALLEN Zellen keine Werte haben
data_1_no_rows = raw_data_1.dropna(how='all')
data_2_no_rows = raw_data_2.dropna(how='all')

# Alle Kolonen löschen die NA Werte enthalten
data_1_no_na = data_1_no_rows.dropna(axis= 'columns')
data_2_no_na = data_2_no_rows.dropna(axis= 'columns')

In [ ]:
#Um eine Vergleichbarkeit von Daten herstellen zu können sollte die Benennung der Zellen die selbe sein
data_stelle_1= data_1_no_na.rename(columns= {'Datum':'Date',
                                             'Uhrzeit': 'Time',
                                             'Masse [kg]' : 'Masse in kg' ,
                                             'Geschwindigkeit [m/s]' : 'Speed in m/s' 
                                            })

data_stelle_2= data_2_no_na.rename(columns= {'Uhrzeit': 'Time',
                                             'm [kg]' : 'Masse in kg' ,
                                             'v [m/s]' : 'Speed in m/s' 
                                            })

In [ ]:
# Überprüfen wie viele "NA" Werte noch vorhanden sind nachdem auch die Anpassung der Benennungen durchgeführt worden ist
data_stelle_1.isna().sum()

In [ ]:
data_stelle_2.isna().sum()

In [ ]:
# Zeit und Datum in eine Spalte bringen und in ein Datetime Object transformieren
data_stelle_1 ['Datetime'] = pd.to_datetime(data_stelle_1 ['Date'] + " " + data_stelle_1 ['Time'])
data_stelle_2 ['Datetime'] = pd.to_datetime(data_stelle_2 ['Date'] + " " + data_stelle_2 ['Time'])

In [ ]:
# Die beiden Spalten 'Date' und 'Time' werden nicht mehr benötigt und werden gelöscht
data_stelle_1 = data_stelle_1.drop('Time', axis=1)
data_stelle_1 = data_stelle_1.drop('Date', axis=1)

data_stelle_2 = data_stelle_2.drop('Time', axis=1)
data_stelle_2 = data_stelle_2.drop('Date', axis=1)

In [ ]:
# Datetime nach Zeit sortieren
data_stelle_2.sort_values(by= ['Datetime'])
data_stelle_1.sort_values(by= ['Datetime'])

## 2. Hinzufügen und Berechnung von fehlenden Daten

In [ ]:
# Berechnung der Zeit zwischen den einzelnen Steinschlägen. 
# Die Anzahl Differenzen beläuft sich auf Anzahl Beobachtungen - 1, da der Startpunkt nicht berücksichtig werden kann

# Zeitdifferenzen Abschlagstelle 1
t_delta_stelle_1 = []
for x in range (len(data_stelle_1['Datetime'])):
    try:
        delta = int(abs(data_stelle_1 ['Datetime'] [x] - data_stelle_1 ['Datetime'] [x+1]).total_seconds() / 3600)
        t_delta_stelle_1.append(delta)   
    except:
        pass   

# Zeitdifferenzen Abschlagstelle 2
t_delta_stelle_2 = []
for x in range (len(data_stelle_2['Datetime'])):
    try:
        delta = int(abs(data_stelle_2 ['Datetime'] [x] - data_stelle_2 ['Datetime'] [x+1]).total_seconds() / 3600)
        t_delta_stelle_2.append(delta)   
    except:
        pass   
    
# Liste in Pandas DataFrame Objekt transformieren
t_delta_stelle_1 = pd.DataFrame(t_delta_stelle_1, columns = ['Delta'])
t_delta_stelle_2 = pd.DataFrame(t_delta_stelle_2, columns = ['Delta'])

In [ ]:
# Die Masse der Steine muss immer > 0 sein, da die Masse im vornherein 
# geschätzt wurde können 0 KG Steine aufgerundet werden 
data_stelle_2 ['Datetime'] .where(data_stelle_2 ['Masse in kg'] == 0)

# Wie zu sehen ist am 2019-03-10 16:00:00 (Zeile 23) ein Stein mit der Masse 0 Kg ins Netz gefallen. 
# Diese Masse wird auf 1 kg angepasst
data_stelle_2.at [23, 'Masse in kg'] = 1.0

In [ ]:
#Verteilung des Verkehrs über die Zeit. Dafür werden die Daten des Bundesamts für Statstik herangezogen.
#Credit geht hier an Roman Studer, der die Daten vom Bundesamt für Statistik besorgt hat

traffic = pd.read_excel("traffic.xlsx")

# Normierungfaktors um die percentile auf 1 zu bringen
traffic_factor = 1 / (traffic['percentile'].sum() / 100) 

# Multiplikation der Kolone um die Normierung zu erhalten
traffic['percentile'] *= traffic_factor


## 3. Datenanalyse - Welche Verteilung liegt in den einzelnen Datensätzen vor?
Vorgehen: Zuerst werden Histogramme der einzelnen Variabeln erstellet um einen ersten Rückschluss auf die Verteilung 
zu machen. Ein QQ Plot wird herangezogen um eine zweite Stufe der Analyse durchzuführen.

### Histogramme

In [ ]:
# Erstelle die Histogramme für alle 6 Verteilungen

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(14, 20))
ax0, ax1, ax2, ax3, ax4, ax5 = axes.flatten()


# Masse
ax0.hist(data_stelle_1['Masse in kg'], 
         bins = 20, density = True, 
         histtype = 'bar', color = 'forestgreen',
         edgecolor='white', linewidth=0.4,
         label = 'Masse 1' )

ax1.hist(data_stelle_2['Masse in kg'], 
         bins = 20, density = True, 
         histtype = 'bar', color = 'forestgreen',
         edgecolor='white', linewidth=0.4,
         label = 'Masse 2' )

# Geschwindigkeit
ax2.hist(data_stelle_1['Speed in m/s'], 
         bins = 22, density = True, 
         histtype = 'bar', color = 'skyblue',
         edgecolor='white', linewidth=0.4,
         label = 'Speed 1' )

ax3.hist(data_stelle_2['Speed in m/s'], 
         bins = 22, density = True, 
         histtype = 'bar', color = 'skyblue',
         edgecolor='white', linewidth=0.4,
         label = 'Speed 2' )

# Zeitdelta
ax4.hist(t_delta_stelle_1 ['Delta'], 
         bins = 50, density = True, 
         histtype = 'bar', color = 'darksalmon',
         edgecolor='white', linewidth=0.4,
         label = 'Zeitdelta 1' )

ax5.hist(t_delta_stelle_2 ['Delta'], 
         bins = 15, density = True, 
         histtype = 'bar', color = 'darksalmon',
         edgecolor='white', linewidth=0.4,
         label = 'Zeitdelta 2' )


ax0.set_title("Histogramm der Masse - Stelle 1")
ax0.set_ylabel("Anzahl Ablösungen (normiert)")
ax0.set_xlabel("Masse in kg") 
ax0.spines['top'].set_visible(False)
ax0.spines['right'].set_visible(False)
ax0.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)

ax1.set_title("Histogramm der Masse - Stelle 2")
ax1.set_ylabel("Anzahl Ablösungen (normiert)")
ax1.set_xlabel("Masse in kg")
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)

ax2.set_title("Histogramm der Geschwindigkeit - Stelle 1")
ax2.set_ylabel("Anzahl Ablösungen (normiert)")
ax2.set_xlabel("Geschwindigkeit in m/s")
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)

ax3.set_title("Histogramm der Geschwindigkeit - Stelle 2")
ax3.set_ylabel("Anzahl Ablösungen (normiert)")
ax3.set_xlabel("Geschwindigkeit in m/s")
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)

ax4.set_title("Histogramm des Zeitdeltas - Stelle 1")
ax4.set_ylabel("Anzahl Ablösungen (normiert)")
ax4.set_xlabel("Zeitdelta in Stunden")
ax4.spines['top'].set_visible(False)
ax4.spines['right'].set_visible(False)
ax4.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)

ax5.set_title("Histogramm des Zeitdeltas - Stelle 1")
ax5.set_ylabel("Anzahl Ablösungen (normiert)")
ax5.set_xlabel("Zeitdelta in Stunden")
ax5.spines['top'].set_visible(False)
ax5.spines['right'].set_visible(False)
ax5.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)

fig.tight_layout()
plt.show()

### Erkenntnis

Nachdem ersten Einblick in die Daten und Histogramme ist zu sehen das wir mehrere Verteilungen erwarten können. 
Bei der Geschwindigkeit wird es sich höchstwahrscheinlich um eine normalverteilte Verteilungen handeln, 
wohingegen bei den anderen weitere Untersuchungen durchgeführt werden müssen um Sicherheit zu gewinnen.

In [ ]:
# QQ - Plot Geschwindigkeit Stelle 1

plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(data_stelle_1['Speed in m/s'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Speed in m/s Stelle 1")
plt.grid(True)
plt.show()

In [ ]:
# QQ - Plot Geschwindigkeit Stelle 2

plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(data_stelle_2['Speed in m/s'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Speed in m/s Stelle 2")
plt.grid(True)
plt.show()

### Erkenntnis Geschwindigkeit
Bei der Gecshwindigkeit 1 ist schön zu beobachten, dass es keine nennenswerte Abweichungen auf dem QQ Plot zu 
erkennen gibt, damit kann die Hypothese von weiter oben, dass es sich um eine Normalverteilung handelt, weiterhin beibehalten werden. 
Im Fall der Geschwindigkeit 2 sollte definitiv noch einmal tiefer ins Detail gegangen werden, da die Abweichungen grösser als die erwarteten bei einer Normalverteilung sind.


In [ ]:
# QQ - Plot Masse Stelle 1

plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(data_stelle_1['Masse in kg'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Msse in kg Stelle 1")
plt.grid(True)
plt.show()

In [ ]:
# QQ - Plot Masse Stelle 2

plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(data_stelle_2['Masse in kg'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Masse in kg Stelle 2")
plt.grid(True)
plt.show()

### Erkenntnis Masse
Wie weiter oben schon erkannt, handelt es sich hier definitiv nicht um eine Normalverteilung.
Was weiter noch erkennbar ist: 
- die Werte sind nicht <0
- rechtsschief
- heavy tails (schwere Ränder) nach oben
- starke Konzentration um den Median (Stelle 1)


In [ ]:
# QQ - Plot Zeitdelta Stelle 1

plt.figure(figsize=(8, 8), dpi=80)
stats.probplot(t_delta_stelle_1['Delta'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Zeitdelta Stelle 1")
plt.grid(True)
plt.show()

In [ ]:
# QQ - Plot Zeitdelta Stelle 2

plt.figure(figsize=(6, 6), dpi=80)
stats.probplot(t_delta_stelle_2['Delta'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Zeitdelta Stelle 2")
plt.grid(True)
plt.show()

### Erkenntnis Zeitdelta
- rechtsschief
- heavy tails (schwere Ränder) - noch extremer als bei der Geschwindigkeit
- Gaps bei Zeitdelta 1 beim 1. Quantil

### Weiteres Vorgehen
Da es sich nicht eindeutig definieren lässt welche Verteilungen zugrunde liegen werden im nächsten Schritt 80 verschiedene Verteilungen auf ihre mögliche Passgenauigkeit überprüft. Das Beurteilungskrieterium ist der statistische Fehler. Je kleiner der Fehler dest besser passt die Verteilung.


## 4. Fitting der Daten
Nach dem Fitten sollte für jeden Datensatz eine Verteilung mit den dazugehörigen Parametern verfügbar sein.

In [ ]:
# Fitten der Daten Masse Ablösestelle 1
fit_masse_1 = Fitter(data_stelle_1['Masse in kg'], distributions=['beta','lognorm', 
                                                                  'gamma', 'expon',
                                                                  'norm'])

fit_masse_1.fit()

In [ ]:
plt.figure(figsize=(8, 8), dpi=80)
fit_masse_1.summary()

In [ ]:
# Fitten der Daten Masse Ablösestelle 2
fit_masse_2 = Fitter(data_stelle_2['Masse in kg'], distributions=['lognorm', 'gamma', 'expon',
                                                                  'beta', 'triang', 'norm'])


fit_masse_2.fit()

In [ ]:
plt.figure(figsize=(8, 8), dpi=80)
fit_masse_2.summary()

In [ ]:
# Fitten der Daten Geschwindigkeit Ablösestelle 1
fit_speed_1 = Fitter(data_stelle_1['Speed in m/s'], distributions=['lognorm', 'gamma', 'expon',
                                                                  'triang', 'norm'])

fit_speed_1.fit()

In [ ]:
plt.figure(figsize=(8, 8), dpi=80)
fit_speed_1.summary()

In [ ]:
# Fitten der Daten Geschwindigkeit Ablösestelle 2
fit_speed_2 = Fitter(data_stelle_2['Speed in m/s'], distributions=['lognorm', 'gamma', 'expon',
                                                                  'triang', 'norm'])


fit_speed_2.fit()

In [ ]:
plt.figure(figsize=(8, 8), dpi=80)
fit_speed_2.summary()

In [ ]:
# Fitten der Daten Zeitdelta Ablösestelle 1
fit_t_delta_1 = Fitter(t_delta_stelle_1['Delta'], distributions=['lognorm', 'gamma', 'expon',
                                                                  'triang', 'norm'])

fit_t_delta_1.fit()

In [ ]:
plt.figure(figsize=(8, 8), dpi=80)
fit_t_delta_1.summary()

In [ ]:
# Fitten der Daten Zeitdelta Ablösestelle 2
fit_t_delta_2 = Fitter(t_delta_stelle_2['Delta'], distributions=['lognorm', 'gamma', 'expon',
                                                                  'triang', 'norm'])

fit_t_delta_2.fit()

In [ ]:
plt.figure(figsize=(8, 8), dpi=80)
fit_t_delta_2.summary()

## 5. Zufalsvariabeln definieren
Nachdem die Art der Verteilungen eruiert worden sind, kann im nächsten Schritt ein Zufallsvariabelngenerator definiert werden als Vorbereitung für die Monte Carlo Simulation


In [ ]:
fit_masse_1.fitted_param['expon']

In [ ]:
masse_1_shape = 12.0
masse_1_scale = 616.6323529411765

In [ ]:
fit_masse_2.fitted_param['expon']

In [ ]:
masse_2_shape = 1
masse_2_scale = 98.28125

In [ ]:
fit_speed_1.fitted_param['norm']

In [ ]:
speed_1_mu = 8.788235294117646
speed_1_std = 1.9745088303442118

In [ ]:
fit_speed_2.fitted_param['norm']

In [ ]:
speed_2_mu = 37.790625
speed_2_std = 5.31080027956004

In [ ]:
fit_t_delta_1.fitted_param['gamma']

In [ ]:
delta_t_1_shape = 0.7720943018929209
delta_t_1_scale = 32.180388904581854

In [ ]:
fit_t_delta_2.fitted_param['gamma']

In [ ]:
delta_t_2_shape = 0.6924339239082432
delta_t_2_scale = 85.33142881264439

In [ ]:
def random_masse_1():
    masse_1 = round(np.random.exponential(masse_1_scale),2)
    return masse_1

def random_masse_2():
    masse_2 = round(np.random.exponential(masse_2_scale),2)
    return masse_2

def random_speed_1():
    speed_1 = round(np.random.normal(speed_1_mu, speed_1_std),2)
    return speed_1

def random_speed_2():
    speed_2 = round(np.random.normal(speed_2_mu, speed_2_std),2)
    return speed_2

def random_t_delta_1():
    t_delta_1 = round(np.random.gamma(delta_t_1_shape, delta_t_1_scale))
    return t_delta_1

def random_t_delta_2():
    t_delta_2 = round(np.random.gamma(delta_t_2_shape, delta_t_2_scale))
    return t_delta_2

In [ ]:
x = random_speed_1()
print (x)

### Erkenntnisse Fitting
Wie vorausgesagt haben wir einerseits mehrere Vertielungen gesehen, andernseits hat sich auch bestätigt, dass die Gaussische Normalverteilung in den Geschwindigkeiten wiederspiegelt.
Die oben definierten Zufallsgeneratoren für die Zahlen Geschwindigkeit, Masse und Zeitdelta werden in der Monte Carlo Simulation verwendet um 100 Jahre der Simulation durchführen zu können.

DISCLAIMER I: SciPy hat 80 verschiedene Distributionen die getestet werden können. aus Komplexitätsgründen wurde dabei nur ein Teil ausgewählt.
DISCLAIMER II: Die Beschriftung der Fitting Outputs (Plot mit Histogramm und Probability Density Function) war in diesem Fall nicht möglich, Ziel wäre es gewesen alle Plots zu beschriften. Dieses Ziel konnte hier nicht erreich werden.
  

## 6. Monte Carlo berechnen

In [ ]:
header = ['Hours', 'Day', 'M 1', 'Speed 1', 'M 2', 'Speed 2', 'E in kJ 1', 'E in kJ 2', 'Sum M', 'Breakthrough']
all_breakthroughs = pd.DataFrame( columns = header )

for x in range (100):
    
    # Kreieren von Leeren Listen um die Resultate zu sammeln
    results = []
    double_entries = []

    # Definition der Betrachtungsdauer für die Monte Carlo
    # 100 Jahre * 365 Tage * 24 Stunden = 8760 Stunden

    amount_of_hours = 876000

    # Kreiere eine Liste um die die Resultate darin abzufüllen
    for i in range (0, amount_of_hours, 1):
        results.append(i)

    for i in range (0, amount_of_hours, 1):
        double_entries.append(i)

    # Transformation in ein Pandas DataFrame Object    
    results = pd.DataFrame(results, columns= ['Hours'])
    double_entries = pd.DataFrame(double_entries, columns= ['Hours'])

    # Definition der benötigten Spalten
    header = ['Day', 'Hour of Day', 'M 1', 'Speed 1', 'M 2', 'Speed 2', 'E in kJ 1', 'E in kJ 2', 'Sum M', 'Breakthrough']

    # Hinzufügen aller Spalten ins DataFrame Object und befülle mit NaN Werten
    for value in header:
        results[value] = np.nan

    for value in header: 
        double_entries[value] = np.nan

    # Start- und Endpunkt für die Monte Carlo Simulation
    end = amount_of_hours
    start = 0

    # Iterationsschritt für Masse und Geschwindigkeit
    i = 0
    j = 0

    # Fortlaufender Zähler der Zeit des Abschlags
    event_1 = 0
    event_2 = 0


    # Berechnungen der Zufallsvariablen in der Monte Carlo Simulation
    # Im ersten Schritt wird das Zeitdelta berechnet, ist das Zeitdelta > 0 dann wird die Masse und die Geschwindigkeit 
    # als Zufallsvariable berechnet. Dabei wird die Funktion von weiter oben abgerufen. Im letzten Schritt werden Masse 
    # und Geschwindigkeit zum Zeitpunkt "event_X" im DataFrame Objekt gespeichert. 

    # Kommt es dazu, dass ein Event zur gleichen Zeit zwei Mal vorkommt (2 Abschläge im gleichen Zeitenster, dann wird dies im 
    # DataFrame "double_entries" abgespeichert (Bedingung t_delta_X = 0)

    while i < end:
        t_delta_1 = random_t_delta_1()

        if t_delta_1 > 0:        
            event_1 += t_delta_1

            masse_1 = random_masse_1()
            results['M 1'][event_1] = masse_1

            speed_1 = random_speed_1()
            results['Speed 1'][event_1] = speed_1
            i += t_delta_1

        else: 
            masse_1 = random_masse_1()
            double_entries['M 1'][event_1] = masse_1

            speed_1 = random_speed_1()
            double_entries['Speed 1'][event_1] = speed_1

    while j < end:
        t_delta_2 = random_t_delta_2()

        if t_delta_2 > 0:
            event_2 += t_delta_2

            masse_2 = random_masse_2()
            results['M 2'][event_2] = masse_2

            speed_2 = random_speed_2()
            results['Speed 2'][event_2] = speed_2
            j += t_delta_2

        else:
            masse_2 = random_masse_2()
            double_entries['M 2'][event_2] = masse_2

            speed_2 = random_speed_2()
            double_entries['Speed 2'][event_2] = speed_2 


    # Da es wenige Ereignisse gibt die im gleichen Zeitfenster geschehen, können die überflüssigen Zeilen rausgefiltert 
    # werden

    filtered_double_entries = double_entries.dropna(thresh = 2)

    # Tabelle zusammenfügen, bereinigen und auffüllen

    frames = [results, filtered_double_entries]
    all_data = pd.concat(frames)

    all_data = all_data.sort_values(by = ['Hours'] )

    all_data = all_data.fillna(0)

    all_data = all_data.reset_index(drop = True)

    # Berechnung fehlenden Felder

    # Formel für kinetische Energie verwenden
    all_data['E in kJ 1'] = all_data.apply(lambda x: all_data['M 1'] * all_data['Speed 1'] * all_data['Speed 1'] / 2 / 1000)
    all_data['E in kJ 2'] = all_data.apply(lambda x: all_data['M 2'] * all_data['Speed 2'] * all_data['Speed 2'] / 2 / 1000)

    # Berechnung des Tages um eine bessere Kotrolle im Enddokument vornehmen zu können
    all_data['Day'] = all_data.apply(lambda x: all_data['Hours'] // 24)

    # Berechnung der Stunde des Tages um eine bessere Kontrolle im Enddokument vornehmen zu können
    all_data['Hour of Day'] = all_data.apply(lambda x: all_data['Hours'] % 24)

    # In diesem Schritt werden die Steine alle 24 Stunden geleert. Um dies zu bewerkstelligen, werden zur Stunde 0 die 
    # Summen in den Netzen geleert und ansonsten immer mit dem vorherigen Element gefüllt.

    INDEX = 0
    end = all_data['Hours'].iloc[-1]
    sum_M = 0

    while INDEX <= end:

        modulo = all_data['Hours'][INDEX] % 24

        if modulo != 0:
            sum_M += all_data['M 1'][INDEX] + all_data['M 2'][INDEX]

            all_data['Sum M'][INDEX] = sum_M

            INDEX += 1

        else:
            sum_M = all_data['M 1'][INDEX] + all_data['M 2'][INDEX]
            all_data['Sum M'][INDEX] = sum_M
            INDEX += 1

    # Im nächsten Schritt wird beurteilt ob der runtergefallene Stein einen Durchbruch im Netz hervorruft. Ist dies der 
    # Fall, dann wird in der Zelle "Breakthrough" eine 1 ( = True) geschrieben, ansonsten eine 0 ( = False)

    INDEX = 0
    end = all_data['Hours'].iloc[-1]

    while INDEX < end:

        if all_data['Hour of Day'][INDEX] == 0:

            if (all_data['E in kJ 1'][INDEX] or all_data['E in kJ 2'][INDEX]) >= 1000:
                all_data['Breakthrough'][INDEX] = 1

                while True:
                    INDEX += 1

                    if all_data['Hour of Day'][INDEX] == 0:
                        break

            else:
                INDEX += 1

        else:
            if all_data['Sum M'][INDEX-1] < 2000:

                if (all_data['E in kJ 1'][INDEX] or all_data['E in kJ 2'][INDEX]) >= 1000:
                    all_data['Breakthrough'][INDEX] = 1

                    while True:
                        INDEX += 1

                        if all_data['Hour of Day'][INDEX] == 0:
                            break

                else:
                    INDEX += 1


            elif all_data['Sum M'][INDEX-1] > 2000:

                if (all_data['E in kJ 1'][INDEX] or all_data['E in kJ 2'][INDEX]) >= 500:
                    all_data['Breakthrough'][INDEX] = 1

                    while True:
                        INDEX += 1

                        if all_data['Hour of Day'][INDEX] == 0:
                            break

                else:
                    INDEX += 1

    # Speichere das Dokument ab
    writer = pd.ExcelWriter('output_per_hour.xlsx') 
    all_data.to_excel(writer)
    writer.save()

    # Kreiere ein Subset der Breakthroughs und speichere das Dokument um es später wieder verwenden zu können

    subset = all_data[(all_data['Breakthrough'] > 0)]
    if len(subset)>0:
        print ('yes')
    else:
        print('no')
        
    all_breakthroughs = all_breakthroughs.append (subset, ignore_index=True)
    
writer = pd.ExcelWriter('all_breakthroughs.xlsx') 
all_breakthroughs.to_excel(writer)
writer.save()

In [ ]:
writer = pd.ExcelWriter('breakthrough.xlsx') 
subset.to_excel(writer)
writer.save()

## 7. Berechnung ob ein Fahrzeug getroffen wird wenn das Netz bricht

Um die Zeit in der Gefahrenzone zu berechnen, wird die Anzahl Fahrzeuge im Zeitfenster mit der Zeit im Zeitfenster 
multipliziert. die Zeit pro Fahrzeug in der Gefahrenzone wird berechnet mit der Geschwindigkeit und der Länge der 
Gefahrenzone. 

In [ ]:
breakthrough = pd.read_excel('breakthrough.xlsx')
verteilung_verkehr = pd.read_excel ('traffic.xlsx')

breakthrough

# In der MonteCarlo Simulation haben wir als Betrcahtungzeitraum 100 Jahre definiert. Dies entspricht 876000 Stunden
Stunden = amount_of_hours

# Aus "breakthrough" brauchen wir nur die Stunde und den %-ualen Aneile der Fahrzeuge um berechnen zu können wie 
# viele Fahrzeuge und wie lange diese in der Gefahrenzone sind.

# Wenn die Gefahrenzone 20 Meter beträgt, dann muss betrachtet werden wie lange das Auto in der Gefahrenzone ist. 
# Die Annahme ist, dass bei einem Kontakt von Auto und Stein ein Todesfall verbucht wird.
# Die Länge des Autos ist 5 m


In [ ]:
anz_unfälle = all_breakthroughs.drop(columns = ['Hours', 'Day', 
                                       'M 1', 'Speed 1', 'M 2', 'Speed 2',
                                       'E in kJ 1', 'E in kJ 2', 'Sum M',
                                       'Breakthrough'])

anz_unfälle ['Anz_Fahrz'] = np.nan
anz_unfälle ['t_event_danger'] = np.nan
anz_unfälle ['Todesfall'] = np.nan
verteilung_verkehr ['Anz_Fahrz'] = np.nan



In [ ]:
länge_gefahrenzone = 0.300
fahrzeug_speed = 60
total_fahrzeuge_tag = 1200

t_danger_zone = länge_gefahrenzone / fahrzeug_speed

verteilung_verkehr['Anz_Fahrz'] = verteilung_verkehr.apply(lambda x: verteilung_verkehr['percentile'] / 100 *  1200)

verteilung_verkehr.Anz_Fahrz = verteilung_verkehr.Anz_Fahrz.round()
verteilung_verkehr

In [ ]:
anz_unfälle

In [ ]:
# Basierend auf der Stunde des Durchbruchs wird die Anzahl der Fahrzeuge in dieser Stunde hinzugefügt
try:
    for x in range (0,len(anz_unfälle)):
        anz_unfälle ['Anz_Fahrz'][x] = verteilung_verkehr['Anz_Fahrz'][(anz_unfälle['Hour of Day'][x])]

# Mit der Anzahl Fahrzeuge kann berechnet werden wie lange Fahrzeuge in der Gefahrenzone bleiben
        anz_unfälle['t_event_danger'] = anz_unfälle.apply(lambda x: anz_unfälle['Anz_Fahrz'] * t_danger_zone)
    
except:
    print ("Es besteht keine Möglichkeit eines Netzdurchbruchs")
    exit()

In [ ]:
pd.set_option('display.max_rows', anz_unfälle.shape[0]+1)
print(anz_unfälle)

In [ ]:
def random_time():
    random_time = random.random()
    return random_time

In [ ]:
start = 0
end = len(anz_unfälle)
TOT = 0
for i in range(end):
    event_time = random_time()
    print (" event %.3f" % event_time)
    if event_time > anz_unfälle['t_event_danger'][i]:
        print (" danger %.3f" % anz_unfälle['t_event_danger'][i] )
        anz_unfälle['Todesfall'] = 0
    
    else:
        anz_unfälle['Todesfall'] = 1
        print ("tot")
        TOT += 1
    
print (TOT)
        


In [ ]:
# Die Wahrscheinlichkeit für einen Todesfall resultiert aus den Anz. Todesfällen / Anz. Betrachtungen

P_Todesfall = TOT / Stunden

if P_Todesfall > 0.0001:
    print('Die Strasse wird gesperrt')
else:
    print('Die Strasse wird NICHT gesperrt')
print (P_Todesfall)